TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [75]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [76]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [77]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 60

In [78]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [79]:
corpus[0:500]

"Good morning. How are you?  Good. It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.  There have been three themes running through the conference, which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here; just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen in te"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [80]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '%', '.', '0', '1', '2', '3', '4', '5', '6', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’', '…', '♫']


In [81]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [82]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 66


In [83]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '%': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '8': 11, '9': 12, '?': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'Y': 36, 'a': 37, 'b': 38, 'c': 39, 'd': 40, 'e': 41, 'f': 42, 'g': 43, 'h': 44, 'i': 45, 'j': 46, 'k': 47, 'l': 48, 'm': 49, 'n': 50, 'o': 51, 'p': 52, 'q': 53, 'r': 54, 's': 55, 't': 56, 'u': 57, 'v': 58, 'w': 59, 'x': 60, 'y': 61, 'z': 62, '’': 63, '…': 64, '♫': 65}


In [84]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '%', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '8', 12: '9', 13: '?', 14: 'A', 15: 'B', 16: 'C', 17: 'D', 18: 'E', 19: 'F', 20: 'G', 21: 'H', 22: 'I', 23: 'J', 24: 'K', 25: 'L', 26: 'M', 27: 'N', 28: 'O', 29: 'P', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'Y', 37: 'a', 38: 'b', 39: 'c', 40: 'd', 41: 'e', 42: 'f', 43: 'g', 44: 'h', 45: 'i', 46: 'j', 47: 'k', 48: 'l', 49: 'm', 50: 'n', 51: 'o', 52: 'p', 53: 'q', 54: 'r', 55: 's', 56: 't', 57: 'u', 58: 'v', 59: 'w', 60: 'x', 61: 'y', 62: 'z', 63: '’', 64: '…', 65: '♫'}


In [85]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 45,560 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [86]:
corpus[0:50]

'Good morning. How are you?  Good. Its been great h'

In [87]:
corpus[1:41]

'ood morning. How are you?  Good. Its bee'

In [88]:
corpus[41]

'n'

In [89]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [90]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[20, 51, 51, 40, 0, 49, 51, 54, 50, 45, 50, 43, 3, 0, 21, 51, 59, 0, 37, 54, 41, 0, 61, 51, 57, 13, 0, 0, 20, 51, 51, 40, 3, 0, 22, 56, 55, 0, 38, 41, 41, 50, 0, 43, 54, 41, 37, 56, 0, 44, 37, 55, 50, 56, 0, 45, 56, 13, 0, 22]
[51, 51, 40, 0, 49, 51, 54, 50, 45, 50, 43, 3, 0, 21, 51, 59, 0, 37, 54, 41, 0, 61, 51, 57, 13, 0, 0, 20, 51, 51, 40, 3, 0, 22, 56, 55, 0, 38, 41, 41, 50, 0, 43, 54, 41, 37, 56, 0, 44, 37, 55, 50, 56, 0, 45, 56, 13, 0, 22, 58]


In [91]:
print ( y_numeric_list[0:2])

[58, 41]


In [92]:
indices_char[51]

'o'

In [93]:
corpus[0:43]

'Good morning. How are you?  Good. Its been '

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [94]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 45,500 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [95]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [96]:
X.shape

(45500, 60, 1)

In [97]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [98]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [99]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [100]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [101]:

checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)
latest

In [102]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


# #model = load_model(f'{latest}')

model = load_model(f'{root_path}models/ted_model_{tag_name}')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 500)           1004000   
_________________________________________________________________
dropout (Dropout)            (None, 60, 500)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               322048    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 66)                8514      
Total params: 1,334,562
Trainable params: 1,334,562
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [103]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 38867 sequence.
[47, 50, 51, 59, 0, 59, 44, 51, 0, 55, 52, 51, 50, 55, 51, 54, 41, 40, 0, 56, 44, 45, 55, 0, 54, 41, 55, 41, 37, 54, 39, 44, 13, 0, 32, 44, 41, 0, 19, 41, 40, 41, 54, 37, 48, 0, 30, 41, 55, 41, 54, 58, 41, 0, 15, 37, 50, 47, 0, 51]


Convert this back to text so we can see what it looks like.

In [104]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'know who sponsored this research? The Federal Reserve Bank o'

In [105]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

know who sponsored this research? The Federal Reserve Bank o
now who sponsored this research? The Federal Reserve Bank of
ow who sponsored this research? The Federal Reserve Bank of 
w who sponsored this research? The Federal Reserve Bank of t
 who sponsored this research? The Federal Reserve Bank of th
who sponsored this research? The Federal Reserve Bank of the
ho sponsored this research? The Federal Reserve Bank of the 
o sponsored this research? The Federal Reserve Bank of the U
 sponsored this research? The Federal Reserve Bank of the Un
sponsored this research? The Federal Reserve Bank of the Uni
ponsored this research? The Federal Reserve Bank of the Unit
onsored this research? The Federal Reserve Bank of the Unite
nsored this research? The Federal Reserve Bank of the United
sored this research? The Federal Reserve Bank of the United 
ored this research? The Federal Reserve Bank of the United S


To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [106]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 66)
Prediction:
[[1.40310974e-09 1.09730435e-13 5.09492700e-17 6.89060475e-11
  2.66055173e-25 7.20080596e-24 7.95560535e-20 7.39199453e-21
  6.65260462e-22 2.44683008e-25 2.92800707e-22 6.49115493e-17
  3.50844963e-27 3.45919886e-12 7.91903557e-11 4.74704656e-21
  4.93845520e-17 3.53067453e-17 1.31695856e-13 3.03206002e-21
  4.65161626e-18 9.13676176e-17 3.18247951e-13 1.08749495e-24
  7.06302367e-13 4.78695087e-18 1.18106218e-19 1.71556266e-16
  9.70830452e-19 3.18912215e-20 5.15140519e-25 8.42134348e-12
  1.24670531e-16 4.88561880e-23 1.34253461e-23 3.24220548e-16
  1.23137099e-11 2.64712918e-09 6.44333340e-07 8.67039063e-08
  3.84290405e-10 1.75985548e-07 9.92884398e-01 9.52395922e-08
  5.84415102e-05 7.19613809e-07 3.30722453e-16 5.72033798e-08
  3.66795982e-10 6.06320327e-08 6.67728018e-03 2.30800106e-06
  3.25121368e-06 3.18995143e-11 1.30769433e-06 3.44038272e-04
  2.41820053e-05 4.87442122e-08 9.02911211e-07 1.40938016e-06
  7.34056371e-08 4.89309116e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [107]:
np.argmax(prediction)

42

So the first letter predicted after the seed sequence is:

In [108]:
indices_char[np.argmax(prediction)]

'f'

In [109]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(500):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [110]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['know who sponsored this research? The Federal Reserve Bank o->f the Univet Sfinol Toond ands say W poa dilan esotpe this',
 'hised sreathene cinc monkes tha woold is thisge touv in thengs bomass in this eiveredtaly thin they gant trogised to',
 'themr ceaaniet and tom on the mett pigdt fow the sas on toe couule rlints iut  Bnt this time celled coilns of to ae',
 'wrong yo thope te hear torkeil for the say on toe shings arou dulw whow isst amlod  And the girl said Im Wrawing a',
 'sicture of wee foian soint wiin a woule witn the eound sout? cerr iut hentsme thool of edlots']

In [111]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['know who sponsored this research? The Federal Reserve Bank of the United States. Thats the American experience. Lets go',
 'across the pond to the London School of Economics LSE London School of Economics alma mater of eleven Nobel Laureates in',
 'economics. Training ground for great economic thinkers like George Soros and Friedrich Hayek and Mick Jagger.  Last',
 'month just last month economists at LSE looked at 51 studies of payforperformance plans inside of companies. Heres what',
 'they  We find that financial incentives can result in a negative impact on overall performance. There is a mismatch',
 'between what science knows and what business does. And what worries me as we stand here in the rubble of the economic',
 'collapse is that too many organizations are making their decisions their policies about talent and people based on',
 'assumptions that are outdated unexamined and rooted more in folklore than in science. And if we really want to get out',
 'of this economic mess